In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from  xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

In [24]:
all_dataset = pd.read_csv("revised_dataset.csv")

In [26]:
all_dataset

,age,gender,pregnant,tsh_level,t3,tt4,t4u,fti,diagnosis
0,41,F,False,none,none,none,none,none,none
1,36,F,False,none,none,none,none,none,none
2,32,F,False,none,none,none,none,none,none
3,60,F,False,none,none,none,none,none,none
4,77,F,False,none,none,none,none,none,none
...,...,...,...,...,...,...,...,...,...
9106,43,M,False,0.005,3,130,0.84,155,hyperthyroid
9107,60,M,False,0.005,none,155,0.78,199,none
9108,42,F,False,0.005,none,178,0.71,251,hyperthyroid
9109,63,M,False,0.005,1.3,96,0.92,104,none


In [27]:
all_dataset['gender'] = all_dataset['gender'].map({'F': 1, 'M': 0})

In [28]:
for value in set(all_dataset['diagnosis']):
    if(value != 'hypothyroid' and value != 'hyperthyroid'):
        all_dataset['diagnosis'].replace(value,'negative',inplace=True)

In [29]:
all_dataset

,age,gender,pregnant,tsh_level,t3,tt4,t4u,fti,diagnosis
0,41,1.0,False,none,none,none,none,none,negative
1,36,1.0,False,none,none,none,none,none,negative
2,32,1.0,False,none,none,none,none,none,negative
3,60,1.0,False,none,none,none,none,none,negative
4,77,1.0,False,none,none,none,none,none,negative
...,...,...,...,...,...,...,...,...,...
9106,43,0.0,False,0.005,3,130,0.84,155,hyperthyroid
9107,60,0.0,False,0.005,none,155,0.78,199,negative
9108,42,1.0,False,0.005,none,178,0.71,251,hyperthyroid
9109,63,0.0,False,0.005,1.3,96,0.92,104,negative


In [30]:
all_dataset = all_dataset.replace({"t":1,"f":0, "y":1, "n":0, "hypothyroid":1, "negative":0,"hyperthyroid":2, "F":1, "M":0})


age            int64
gender       float64
pregnant      object
tsh_level     object
t3            object
tt4           object
t4u           object
fti           object
diagnosis      int64
dtype: object

In [31]:
all_dataset = all_dataset.replace({True:1,False:0, "y":1, "n":0, "hypothyroid":1, "negative":0,"hyperthyroid":2, "F":1, "M":0})
display(all_dataset.dtypes)

age            int64
gender       float64
pregnant     float64
tsh_level     object
t3            object
tt4           object
t4u           object
fti           object
diagnosis      int64
dtype: object

In [32]:
all_dataset

,age,gender,pregnant,tsh_level,t3,tt4,t4u,fti,diagnosis
0,41,1.0,0.0,none,none,none,none,none,0
1,36,1.0,0.0,none,none,none,none,none,0
2,32,1.0,0.0,none,none,none,none,none,0
3,60,1.0,0.0,none,none,none,none,none,0
4,77,1.0,0.0,none,none,none,none,none,0
...,...,...,...,...,...,...,...,...,...
9106,43,0.0,0.0,0.005,3,130,0.84,155,2
9107,60,0.0,0.0,0.005,none,155,0.78,199,0
9108,42,1.0,0.0,0.005,none,178,0.71,251,2
9109,63,0.0,0.0,0.005,1.3,96,0.92,104,0


In [47]:
all_dataset = all_dataset.replace({True:1,False:0, "y":1, "n":0, "hypothyroid":1, "negative":0,"hyperthyroid":2, "F":1, "M":0})
all_dataset=all_dataset.replace({"none":np.NAN})

In [34]:
all_dataset.isna().sum()

age             0
gender        305
pregnant      305
tsh_level     842
t3           2588
tt4           440
t4u           806
fti           799
diagnosis       0
dtype: int64

In [35]:
all_dataset

,age,gender,pregnant,tsh_level,t3,tt4,t4u,fti,diagnosis
0,41,1.0,0.0,NaN,NaN,NaN,NaN,NaN,0
1,36,1.0,0.0,NaN,NaN,NaN,NaN,NaN,0
2,32,1.0,0.0,NaN,NaN,NaN,NaN,NaN,0
3,60,1.0,0.0,NaN,NaN,NaN,NaN,NaN,0
4,77,1.0,0.0,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...
9106,43,0.0,0.0,0.005,3,130,0.84,155,2
9107,60,0.0,0.0,0.005,NaN,155,0.78,199,0
9108,42,1.0,0.0,0.005,NaN,178,0.71,251,2
9109,63,0.0,0.0,0.005,1.3,96,0.92,104,0


In [36]:
cols = all_dataset.columns[all_dataset.dtypes.eq('object')]
all_dataset[cols] = all_dataset[cols].apply(pd.to_numeric, errors='coerce')
display(all_dataset.dtypes)

age            int64
gender       float64
pregnant     float64
tsh_level    float64
t3           float64
tt4          float64
t4u          float64
fti          float64
diagnosis      int64
dtype: object

In [37]:
all_dataset.isna().sum()

age             0
gender        305
pregnant      305
tsh_level     842
t3           2588
tt4           440
t4u           806
fti           799
diagnosis       0
dtype: int64

In [38]:
corr_values = abs(all_dataset[all_dataset.columns[0:]].corr()['diagnosis'][:])
corr_values = corr_values.drop('diagnosis')
corr_values = corr_values[corr_values > 0.04]
display(corr_values)
display(corr_values.index)

gender       0.064354
tsh_level    0.339085
t3           0.241793
tt4          0.132977
fti          0.184833
Name: diagnosis, dtype: float64

Index(['gender', 'tsh_level', 't3', 'tt4', 'fti'], dtype='object')

In [39]:
def holdout(dataframe):
  x = dataframe[corr_values.index]
  y = dataframe['diagnosis']
  X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42) 
  return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = holdout(all_dataset)

In [40]:
classifiers = {
    "XGBClassifier" : XGBClassifier(learning_rate=0.01),
    "CatBoostClassifier" : CatBoostClassifier(max_depth=4,verbose=0),
}

In [41]:
def classification(classifiers, X_train, X_test, y_train, y_test):
    # Creo un dataframe per visualizzare i risultati calcolati
  res = pd.DataFrame(columns=["Classifier", 
                                "Accuracy", 
                                "Precision", 
                                "Recall", 
                                "FScore"])
  for name, clf in classifiers.items():
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            pr, rc, fs, sup = metrics.precision_recall_fscore_support(y_test, y_pred, average='macro')
            res = res.append({"Classifier": name,"Accuracy": round(metrics.accuracy_score(y_test, y_pred), 4),
                              "Precision": round(pr, 4), "Recall":round(rc, 4), "FScore":round(fs, 4)}, ignore_index=True)
            print("Confusion matrix for: ", name)
            display(confusion_matrix(y_test, y_pred))
  res.set_index("FScore", inplace=True)
  res.sort_values(by="FScore", ascending=False, inplace=True)   
  return res

display(classification(classifiers, X_train, X_test, y_train, y_test))

C:\Users\Asus\Anaconda3\envs\py3.7-TF2.0\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:02:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Confusion matrix for:  XGBClassifier


array([[2398,   61,    9],
       [   7,  173,    0],
       [  47,    0,   39]], dtype=int64)

Confusion matrix for:  CatBoostClassifier


array([[2393,   57,   18],
       [  15,  165,    0],
       [  40,    0,   46]], dtype=int64)

,Classifier,Accuracy,Precision,Recall
FScore,,,,
0.8026,CatBoostClassifier,0.9525,0.8132,0.8071
0.7975,XGBClassifier,0.9546,0.8433,0.7954


In [42]:
display(all_dataset.shape)
all_dataset.diagnosis.value_counts()

(9111, 9)

0    8261
1     609
2     241
Name: diagnosis, dtype: int64

In [43]:
data_alt = all_dataset.interpolate(method = 'spline', order = 3,limit_direction='both')
display(data_alt.isna().sum())

age          0
gender       0
pregnant     0
tsh_level    0
t3           0
tt4          0
t4u          0
fti          0
diagnosis    0
dtype: int64

In [44]:
classifiers1 = {
    "XGBClassifier" : XGBClassifier(learning_rate=0.01),
    "CatBoostClassifier" : CatBoostClassifier(max_depth=4,verbose=0),
    "Nearest Neighbors" : KNeighborsClassifier(4),
    "Decision Tree" : DecisionTreeClassifier(class_weight = 'balanced'),
    "Random Forest": RandomForestClassifier(class_weight = 'balanced',random_state = 1),
    "ExtraTrees": ExtraTreesClassifier(class_weight = 'balanced',random_state = 1),
    "MLPClassifier": MLPClassifier(hidden_layer_sizes=(256,128,64,32),activation="relu",random_state=1)
}

In [45]:
corr_values = abs(data_alt[data_alt.columns[0:]].corr()['diagnosis'][:])
corr_values = corr_values.drop('diagnosis')
corr_values = corr_values[corr_values > 0.04]
display(corr_values)

gender    0.064545
t3        0.219407
tt4       0.043935
fti       0.113227
Name: diagnosis, dtype: float64

In [46]:

X_train_alt, X_test_alt, y_train_alt, y_test_alt = holdout(data_alt)

display(classification(classifiers1,X_train_alt, X_test_alt, y_train_alt, y_test_alt))

C:\Users\Asus\Anaconda3\envs\py3.7-TF2.0\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:03:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Confusion matrix for:  XGBClassifier


array([[2447,    9,   12],
       [ 145,   35,    0],
       [  55,    0,   31]], dtype=int64)

Confusion matrix for:  CatBoostClassifier


array([[2431,   21,   16],
       [ 140,   40,    0],
       [  47,    0,   39]], dtype=int64)

...

Confusion matrix for:  ExtraTrees


array([[2417,   36,   15],
       [ 141,   38,    1],
       [  52,    0,   34]], dtype=int64)

Confusion matrix for:  MLPClassifier


array([[2435,   12,   21],
       [ 155,   25,    0],
       [  50,    0,   36]], dtype=int64)

,Classifier,Accuracy,Precision,Recall
FScore,,,,
0.6137,CatBoostClassifier,0.9181,0.7645,0.5536
0.6024,Random Forest,0.9173,0.7738,0.5343
0.5837,ExtraTrees,0.9104,0.7065,0.5286
0.5833,XGBClassifier,0.9192,0.8136,0.5155
0.5685,Decision Tree,0.8804,0.5896,0.5522
0.5624,MLPClassifier,0.9129,0.7432,0.5147
0.4607,Nearest Neighbors,0.9056,0.6820,0.4188
